# Creating a RAG with Groq API's LLama 70b as base model and HuggingFace's sentence-transformer as embedding model

In [ ]:
!pip install langchain sentence-transformers langchain-community chromadb langchain-groq langchain-core

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_AI_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

### Step 1 : Create and populate the Vector Database

Step 1.1 : Load the text file using LangChain's TextLoader

In [4]:
## Data infestion for text files
from langchain_community.document_loaders import TextLoader

loader = TextLoader("speech.txt")
text_documents = loader.load()

Step 1.2 : (Optional) : If its data from a website that we need, then we have to scrape the web data. We can do that using Beautiful Soup module from Python.

After scraping the data, we have to load it. This is done using the WebBaseLoader from LangChain

In [ ]:
## Data ingestion using web based loader

from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_path =("https://www.bbc.com/sport/football/articles/cyrrjv3e4zro"), # URL of the webpage
    bs_kwargs=dict(parse_only = bs4.SoupStrainer(
        class_=("zephr_join_beta","ssrcss-181c4hk-SectionWrapper eustbbg0",
        "ssrcss-4rxmy3-PageStack e1mcntqj2","ssrcss-l6cntj-ContentStack e1mcntqj0",
        "ssrcss-irv5dn-Zone e1mcntqj4","ssrcss-irv5dn-Zone e1mcntqj4",
        "ssrcss-1ocoo3l-Wrap e42f8511","ssrcss-1y7k614-FooterStack e1mcntqj1")
        )
    )
)

web_load = loader.load()
print(web_load)

Step 1.3 : (Optional) : Lets imagine that the data we want if from a PDF. We can load the data in the file using PyPDFLoader

In [ ]:
## Data injestion using PDFs

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("jose's_plan.pdf") # Path to the PDF file
docs = loader.load()
docs

### Step 2 : Chunk the data into smaller segments

RecursiveCharacterTextSplitter splits the data into chunks of the specified chunk size and overlap.

In [8]:
## Dividing the text into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 100)
final_docs = text_splitter.split_documents(docs)

### Step 3 : Ready the embedder from HuggingFace

In [ ]:
## Vector embeddings and Vector Store
from langchain_community.embeddings import HuggingFaceEmbeddings
ollama_emb = HuggingFaceEmbeddings() # huggingface embeddings

In [ ]:
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)
r1  # To check if it works

### Step 4 : Create the embeddings of the loaded data and insert it into the Vector DB

In [10]:
from langchain_community.vectorstores import Chroma
from chromadb.errors import InvalidDimensionException       # Store data into chroma db

try:
    db = Chroma.from_documents(final_docs,ollama_emb)
except InvalidDimensionException:
    Chroma().delete_collection()
    db = Chroma.from_documents(final_docs,ollama_emb)

Lets see if the database is ready by querying it

In [ ]:
query = "What is the playstyle of Tottenham Hotspur ?"
result = db.similarity_search(query)
result      ## Similarity search for vector database

### Step 5 : Prepare the LLM's API Endpoint

In [11]:
# import and ready the llm, here, we use groq api
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(temperature=0, model="llama3-70b-8192")

Step 6 : (<b>MOST IMPORTANT PART<b>) : Pre-prompt is declared

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only the context provided.
    Think step by step and provide a detailed answer.
    Just give the final precise answer, no need to explain.
    <context>
    {context}
    </context>
    Question : {input}
    """
    )


Step 7 : Create the chain between the Prompt, LLM and Retriever

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)
retriever = db.as_retriever()
retriever

Step 8 : Query the model :)

In [ ]:
from langchain.chains import create_retrieval_chain

input_question = input("Enter your question : ")


retrieval_chain = create_retrieval_chain(retriever,document_chain)
retrieval_chain.invoke({"input":input_question,
                        "context":retriever})['answer']